In [ ]:
library(RBGL)
library(ggplot2)
library(dplyr)

source("lib/code_for_binary_simulations/blip_vs_bidag_plot.R")
source("lib/code_for_binary_simulations/summarySE.R")

In [ ]:
directory <- "simresults"

In [ ]:
filenames <- list.files(pattern = "^(res_orderMCMC)", path = directory)
ROCdf_order_mcmc <- data.frame()
for (filename in filenames) {
  tmpdf <- read.csv(file.path(directory, filename))
  ROCdf_order_mcmc <- dplyr::bind_rows(ROCdf_order_mcmc, tmpdf)
}

In [ ]:
ROCdf_order_mcmc

In [ ]:
sumROC_order_mcmc = ROCdf_order_mcmc %>% 
                    group_by(algorithm, threshold, sample_size, dim) %>% 
                    summarise(TPR_mean = mean(TPR), 
                              TPR_median = median(TPR), 
                              FPRn_median = median(FPRn), 
                              TPR_q1 = quantile(TPR, probs = c(0.05)), 
                              TPR_q3 = quantile(TPR, probs = c(0.95)),
                              time = mean(time),
                              logscore_mean = mean(logscore),
                              N = n())

In [ ]:
sumROC_order_mcmc

In [ ]:
filenames <- list.files(pattern = "^(res_pcalg)", path = directory)
roc_pcalg <- data.frame()
for (filename in filenames) {
  tmpdf <- read.csv(file.path(directory, filename))
  roc_pcalg <- dplyr::bind_rows(roc_pcalg, tmpdf)
}

In [ ]:
roc_pcalg

In [ ]:
sum_roc_pcalg <- roc_pcalg %>% 
                 group_by(algorithm, alpha, sample_size, dim) %>% 
                 summarise(TPR_mean = mean(TPR), 
                          TPR_median = median(TPR), 
                          FPRn_median = median(FPRn), 
                          TPR_q1 = quantile(TPR, probs = c(0.05)), 
                          TPR_q3 = quantile(TPR, probs = c(0.95)),
                          N = n()) %>% ungroup()

In [ ]:
sum_roc_pcalg

In [ ]:
filenames <- list.files(pattern = "^(res_mmhc)", path = directory)
roc_mmhc <- data.frame()
for (filename in filenames) {
  tmpdf <- read.csv(file.path(directory, filename))
  roc_mmhc <- dplyr::bind_rows(roc_mmhc, tmpdf)
}

In [ ]:
roc_mmhc

In [ ]:
sum_roc_mmhc <- roc_mmhc %>% 
                 group_by(algorithm, alpha, sample_size, dim) %>% 
                 summarise(TPR_mean = mean(TPR), 
                          TPR_median = median(TPR), 
                          FPRn_median = median(FPRn), 
                          TPR_q1 = quantile(TPR, probs = c(0.05)), 
                          TPR_q3 = quantile(TPR, probs = c(0.95)),
                          N = n()) %>% ungroup()

In [ ]:
sum_roc_mmhc

In [ ]:
filenames <- list.files(pattern = "^(res_itsearch)", path = directory)
ROCdf_itsearch <- data.frame()
for (filename in filenames) {
  tmpdf <- read.csv(file.path(directory, filename))
  ROCdf_itsearch <- dplyr::bind_rows(ROCdf_itsearch, tmpdf)
}

In [ ]:
ROCdf_itsearch

In [ ]:
sum_roc_itsearch <- ROCdf_itsearch %>% 
                 group_by(algorithm, sample_size, dim) %>% 
                 summarise(TPR_mean = mean(TPR), 
                          TPR_median = median(TPR), 
                          FPRn_median = median(FPRn), 
                          TPR_q1 = quantile(TPR, probs = c(0.05)), 
                          TPR_q3 = quantile(TPR, probs = c(0.95)),
                          logscore_mean =  mean(logscore),
                          N = n())

In [ ]:
sum_roc_itsearch

In [ ]:
filenames <- list.files(pattern = "^(res_blip)", path = directory)
ROCdf_blip <- data.frame()
for (filename in filenames) {
  tmpdf <- read.csv(file.path(directory, filename))
  ROCdf_blip <- dplyr::bind_rows(ROCdf_blip, tmpdf)
}

In [ ]:
ROCdf_blip

In [ ]:
sum_roc_blip <- ROCdf_blip %>% 
                 group_by(algorithm, sample_size, dim) %>% 
                 summarise(TPR_mean = mean(TPR), 
                          TPR_median = median(TPR), 
                          FPRn_median = median(FPRn), 
                          TPR_q1 = quantile(TPR, probs = c(0.05)),
                          TPR_q3 = quantile(TPR, probs = c(0.95)),
                          N = n())

In [ ]:
sum_roc_blip

## Plot ROC curves

In [ ]:
setEPS()
postscript(file.path(directory, "ROC.eps"))

ggplot() +
# PC algorithm
geom_errorbar(data = sum_roc_pcalg,
              aes(x = FPRn_median,
                  ymin = TPR_q1, 
                  ymax = TPR_q3, 
                  col = algorithm), 
              width = 0.01) +
geom_path(data = sum_roc_pcalg,
          aes(x = FPRn_median, 
              y = TPR_median,
              col = algorithm)) +
geom_point(data = sum_roc_pcalg,
           aes(x = FPRn_median, 
               y = TPR_median,               
               col = algorithm, 
               shape = algorithm), 
               size = 1) + 
geom_text(data = sum_roc_pcalg,
           aes(x = FPRn_median, 
               y = TPR_q3,               
               label=alpha, col=algorithm),
         check_overlap = TRUE,
          nudge_x=-0.02,
         nudge_y=0.02
         ) +

# Max-min hill- climbing 
geom_errorbar(data = sum_roc_mmhc,
              aes(x = FPRn_median,
                  ymin = TPR_q1, 
                  ymax = TPR_q3, 
                  col = algorithm), 
              width = 0.01) +
geom_path(data = sum_roc_mmhc,
          aes(x = FPRn_median, 
              y = TPR_median,
              col = algorithm)) +
geom_point(data = sum_roc_mmhc,
           aes(x = FPRn_median, 
               y = TPR_median,               
               col = algorithm, 
               shape = algorithm), 
               size = 1) + 
geom_text(data = sum_roc_mmhc,
           aes(x = FPRn_median, 
               y = TPR_q3,               
               label=alpha, col=algorithm),
         check_overlap = TRUE,
          nudge_x=-0.02,
         nudge_y=0.02
         ) +

# Order mcmc
geom_errorbar(data = sumROC_order_mcmc,
              aes(x = FPRn_median,
                  ymin = TPR_q1, 
                  ymax = TPR_q3, 
                  col = algorithm), 
              width = 0.01) +
geom_path(data = sumROC_order_mcmc,
          aes(x = FPRn_median, 
              y = TPR_median,
              col = algorithm)) +
geom_point(data = sumROC_order_mcmc,
           aes(x = FPRn_median, 
               y = TPR_median,               
               col = algorithm, 
               shape = algorithm), 
               size = 2) +
geom_text(data = sumROC_order_mcmc,
           aes(x = FPRn_median, 
               y = TPR_q3,               
               col = algorithm, label=threshold),
           check_overlap = TRUE,
          nudge_x=-0.02,
         nudge_y=0.02) +

# Iterative search
geom_errorbar(data = sum_roc_itsearch,
              aes(x = FPRn_median,
                  ymin = TPR_q1, 
                  ymax = TPR_q3, 
                  col = algorithm), 
              width = 0.01) +
geom_path(data = sum_roc_itsearch,
          aes(x = FPRn_median, 
              y = TPR_median,
              col = algorithm)) +
geom_point(data = sum_roc_itsearch,
           aes(x = FPRn_median, 
               y = TPR_median,               
               col = algorithm, 
               shape = algorithm), 
               size = 1) +
# Blip
geom_errorbar(data = sum_roc_blip,
              aes(x = FPRn_median,
                  ymin = TPR_q1, 
                  ymax = TPR_q3, 
                  col = algorithm), 
              width = 0.01) +
geom_path(data = sum_roc_blip,
          aes(x = FPRn_median, 
              y = TPR_median,
              col = algorithm)) +
geom_point(data = sum_roc_blip,
           aes(x = FPRn_median, 
               y = TPR_median,               
               col = algorithm, 
               shape = algorithm), 
               size = 1) +
## Subplot
facet_wrap(~ dim + sample_size, scales="free") +
# Titles and stuff
xlab("FPRn") +
ylab("TPR") +
ggtitle("ROC") +
theme(plot.title = element_text(hjust = 0.5))
dev.off()


## Logscores

In [ ]:
# Order mcmc
ggplot() + 
geom_path(data = sumROC_order_mcmc,
          aes(x = FPRn_median, 
              y = logscore_mean,
              col = algorithm)) +
geom_point(data = sumROC_order_mcmc,
           aes(x = FPRn_median, 
               y = logscore_mean,               
               col = algorithm, 
               shape = algorithm), 
               size = 2) +
geom_text(data = sumROC_order_mcmc,
           aes(x = FPRn_median, 
               y = logscore_mean,               
               col = algorithm, label=threshold))

In [ ]:
# Order mcmc
ggplot() + 
geom_path(data = sumROC_order_mcmc,
          aes(x = threshold, 
              y = logscore_mean,
              col = algorithm)) +
geom_point(data = sumROC_order_mcmc,
           aes(x = threshold, 
               y = logscore_mean,               
               col = algorithm, 
               shape = algorithm), 
               size = 2) +
geom_text(data = sumROC_order_mcmc,
           aes(x = threshold, 
               y = logscore_mean,               
               col = algorithm, 
               label=threshold))